# IMPORTAÇÕES TRATAMENTO

In [ ]:
import pandas as pd
import sidrapy

# OBTENDO TABELA SIDRAPY

In [ ]:
pop = sidrapy.get_table(
    table_code='4709',               # t (table_code) - é o codigo da tabela no site da sidra IBGE;
    territorial_level='6',           # n (territorial_level) - especifica os niveis territoriais;
    ibge_territorial_code='all',     # n/ (ibge_territorial_code) - inserido dentro do nivel territorial, especificar o codigo territorial do IBGE;
    period='all',                    # p (period) - utilizado para especificar o periodo;
    variable='all',                  # v (variable) - para especificar as variaveis desejadas;
)

print(pop.shape)
pop.head()

# TRATAMENTO

In [ ]:
# Removendo acentos para facilitar o estudo
pop = pop.apply(lambda x: x.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8') if x.dtype == 'object' else x) # EsSa função é aplicada em todas as linhas do tipo object do data frame
# Removendo primeira linha
pop = pop.iloc[1:] # removemos a linha com indice zero que não é uma legenda dos dados

pop.head()

In [ ]:
# Converta a coluna 'V' para numérico, forçando erros a serem convertidos para NaN
pop['V'] = pd.to_numeric(pop['V'], errors='coerce')

In [ ]:
# Pivotando tabela
pop = pop.pivot_table(index=['D1C', 'D1N'], columns ='D3N', values = 'V').reset_index()
pop.columns.name = None

print(pop.shape)
pop.head()

In [ ]:
# Dicionário para mapeamento das colunas
column_mapping = {
    'D1C': 'CD_MUN',
    'D1N': 'NM_MUN',
    'Populacao residente': 'POP_RESIDENTE',
    'Variacao absoluta da populacao residente 2010 compatibilizada': 'VAR_ABS',
    'Taxa de crescimento geometrico': 'TX_CRESC'
}

# Renomeando as colunas
pop.rename(columns = column_mapping, inplace=True)
pop.head()

In [ ]:
# Checando se as os data types estão corretos
pop.info() 

In [ ]:
# Tratando os tipos das colunas, devemos deixar a tx_cresc como float, pop_residente, cd_mun e var_abs como inteiro
pop[['CD_MUN','POP_RESIDENTE','VAR_ABS']] = pop[['CD_MUN','POP_RESIDENTE','VAR_ABS']].astype(int)
pop['TX_CRESC'] = pop['TX_CRESC'].astype(float)

pop.head()

In [ ]:
# Checando se as os data types estão corretos
pop.info() 

# IMPORTAÇÕES VISUALIZAÇÃO

In [ ]:
import matplotlib.pyplot as plt
import plotly.express as px
import wget
import json
from urllib.request import urlopen

In [ ]:
# Lendo o arquivo JSON com a delimitação de cada municipio para o plot do gráfico

with urlopen('https://raw.githubusercontent.com/fea-dev-usp/IBGE/master/geojson_2022.json') as response:
    geo_json = json.load(response)

In [ ]:
# Plotanto o mapa

fig = px.choropleth_mapbox(
    data_frame = pop,
    geojson = geo_json,
    locations = 'CD_MUN',
    featureidkey = 'properties.codarea',
    color = 'TX_CRESC',
    color_continuous_scale = 'thermal',
    range_color = (pop['TX_CRESC'].min(),pop['TX_CRESC'].max()),
    mapbox_style = 'open-street-map',
    zoom = 3.5,
    center = {"lat": -15.81, "lon": -47.93},
    opacity = 1,
    labels = {'TX_CRESC': 'Tx Cresc Geométrico (%)', 'CD_MUN': 'Código do município'},
    width = 1200,
    height = 800,
    title = 'Variação Pop Censo 2022',
    hover_name = 'NM_MUN'
)

# Customizando o layout
fig.update_layout(
    margin={'r': 0, 't': 1, 'l': 0, 'b': 1},
    coloraxis_colorbar={
        'title': {
            'text': 'Taxa de Crescimento (%)',
            'side': 'right'
        }
    },
    showlegend=True,  # Garante que a legenda seja exibida
    legend=dict(
        x=1,  # Posição horizontal (0-1)
        y=1,  # Posição vertical (0-1)
        traceorder='normal',
        orientation='v'  # Vertical ou horizontal
    )
)

# Atualizando as configurações geográficas
fig.update_geos(
    showcoastlines=False,
    visible=False
)

# Atualizando as configurações de traços
fig.update_traces(
    marker_line_width=0.05,
    selector=dict(type='choroplethmapbox')
)

# Exibindo o gráfico
fig.show()